In [10]:
from tool import *
import plotly.express as px
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment, Font, Border, Side
from openpyxl.utils import get_column_letter
from openpyxl.drawing.image import Image
import pyautogui

## GSCP Sales PSI Raw data를  DB에 저장

In [11]:
filepath = get_filename()
created_at = '2024013016'
planweek = get_weekname(datetime.datetime(2024, 1, 30))
ver = 'Final'

In [12]:
psi = pd.read_excel(filepath, sheet_name=0, keep_default_na=False, na_values=['', 'nan']) # GSCP로부터 금주 SP를 데이타프레임화trim_column_name(sp)
trim_column_name(psi)
psi = psi[psi['Rep From Site'].notnull()].reset_index(drop=True)
psi.insert(0, 'Created_at', created_at) # PSI 생성일
psi.insert(0, 'Ver', ver) # 기준 주차명 설정
psi.insert(0, 'Planweek', planweek) # 기준 주차명 설정
psi = psi.rename(columns={'Site':'To Site', 'Rep From Site':'From Site'})
psi = psi.set_index(['Planweek', 'Created_at', 'Ver', 'Division', 'From Site', 'Region', 'To Site', 'Mapping Model.Suffix', 'Rep PMS', 'Category'])[get_weeklist(psi)]
psi.columns.name = 'Week Name'
psi = psi.stack().replace({0:np.nan}).dropna().astype(int).reset_index()
psi.rename(columns={0:'QTY'}, inplace=True)

In [16]:
with open('D:/Data/ODM PSI.db', 'rb') as f:
    psi_db = pickle.load(f)

In [17]:
psi_db = pd.concat([psi_db, psi]).reset_index(drop=True)

In [18]:
psi_db.shape

(202688, 12)

In [19]:
with open('D:/Data/ODM PSI.db', 'wb') as f:
    pickle.dump(psi_db, f)

# PSI 분석

In [20]:
with open('D:/Data/ODM PSI.db', 'rb') as f:
    psi_db = pickle.load(f)
with open('D:/Data/contact_window.db', 'rb') as f:
    cont = pickle.load(f)
with open('D:/Data/sitemap.db', 'rb') as f:
    site = pickle.load(f)

In [25]:
sf = psi_db[psi_db['Category'] == 'Sales Allocation [R+F]'].reset_index(drop=True)

### 1) ODM 전체로 볼 떄

In [120]:
ver_table = sf.groupby(['Planweek', 'Ver', 'Created_at']).sum('SP').reset_index()
ver_table.iloc[-30:, ]

,Planweek,Ver,Created_at,QTY
0,2024-01-15(W03),Final,2024011509,1045657
1,2024-01-22(W04),Final,20240122,1080224
2,2024-01-29(W05),Main,2024013009,1114778


In [121]:
i1 = 2
i2 = 1

p_wk = sf.groupby(['Planweek', 'Ver', 'Created_at']).sum('SP').reset_index().iloc[i1, :]['Planweek']
ver = sf.groupby(['Planweek', 'Ver', 'Created_at']).sum('SP').reset_index().iloc[i1, :]['Ver']

sf1 = sf.merge(sf.groupby(['Planweek', 'Ver', 'Created_at']).sum('QTY').reset_index().iloc[[i1], :3])
sf2 = sf.merge(sf.groupby(['Planweek', 'Ver', 'Created_at']).sum('QTY').reset_index().iloc[[i2], :3])
sf2 = sf2[sf2['Week Name'] >= sf1['Week Name'].min()]

In [122]:
sf_all = pd.concat([sf1, sf2])
sf_all['Month'] = sf_all['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

In [123]:
sf_all = sf_all[sf_all['Week Name'] >= [get_weekname_from(p_wk, i) for i in range(-3, 1)][0]] # 이번주 기준 3주전부터 Sorting
sf_all = sf_all[sf_all['Month'] <= '2024-05'] # 24년 4월까지만 한정

In [127]:
sf_all[sf_all['To Site'].str.contains('EAI')]

,Planweek,Created_at,Ver,Division,From Site,Region,To Site,Mapping Model.Suffix,Rep PMS,Category,Week Name,QTY,Month
656,2024-01-29(W05),2024013009,Main,MNT,OS_AUO_CN_P,INDIA,EAIL,24GQ50F-B.ATRQ,24GQ50F-BD.CTRVMV,Sales Forecast [R+F],2024-01-22(W04),46,2024-01
657,2024-01-29(W05),2024013009,Main,MNT,OS_AUO_CN_P,INDIA,EAIL,24GQ50F-B.ATRQ,24GQ50F-BD.CTRVMV,Sales Forecast [R+F],2024-01-29(W05),4,2024-02
658,2024-01-29(W05),2024013009,Main,MNT,OS_AUO_CN_P,INDIA,EAIL,24GQ50F-B.ATRQ,24GQ50F-BD.CTRVMV,Sales Forecast [R+F],2024-03-04(W10),150,2024-03
659,2024-01-29(W05),2024013009,Main,MNT,OS_AUO_CN_P,INDIA,EAIL,24GQ50F-B.ATRQ,24GQ50F-BD.CTRVMV,Sales Forecast [R+F],2024-03-11(W11),150,2024-03
660,2024-01-29(W05),2024013009,Main,MNT,OS_AUO_CN_P,INDIA,EAIL,24GQ50F-B.ATRQ,24GQ50F-BD.CTRVMV,Sales Forecast [R+F],2024-03-25(W13),272,2024-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
713,2024-01-22(W04),20240122,Final,MNT,OS_TCL_CN_P,INDIA,EAIL,27MR400-B.ATRQ,27MR400-BB.CTRRMV,Sales Forecast [R+F],2024-05-13(W20),360,2024-05
714,2024-01-22(W04),20240122,Final,MNT,OS_TCL_CN_P,INDIA,EAIL,27MR400-B.ATRQ,27MR400-BB.CTRRMV,Sales Forecast [R+F],2024-05-20(W21),420,2024-05
715,2024-01-22(W04),20240122,Final,MNT,OS_TCL_CN_P,INDIA,EAIL,27MR400-B.ATRQ,27MR400-BB.CTRRMV,Sales Forecast [R+F],2024-05-27(W22),75,2024-05
2569,2024-01-22(W04),20240122,Final,PC,OS_QUANTA_CN_P,INDIA,EAIL,16T90R-G.CH78A2,16T90R-G.CH78A2,Sales Forecast [R+F],2024-01-22(W04),15,2024-01


In [172]:
sf_all.insert(7, 'Series', sf_all['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]))
# sf_all.insert(7, 'Inch', sf_all['Mapping Model.Suffix'].apply(lambda x:x[:2]))
sf_all = pd.merge(sf_all, site[['To Site', 'Subsidiary', 'Country']], how='left')

In [202]:
# IT ODM 모델명에서 시리즈명 추출
c1 = sf_all['From Site'].str.contains('AUO|TPV|TCL') # 모니터 ODM 일 경우
c2 = sf_all['Mapping Model.Suffix'].str.match('\d{2}[T|Z|U|A][G|D|N|W|C]?\d{2}[A-Z]{1,2}') # PC/태블릿 모델일 경우
c3 = sf_all['Mapping Model.Suffix'].str.contains('CL600') # 페가트론 CL600 모델일 경우

sf_all.loc[c1, 'Series'] = sf_all[c1]['Mapping Model.Suffix'].apply(lambda x:x[:2]) # 모니터 모델의 시리즈명(inch) 추출
sf_all.loc[c2, 'Series'] = sf_all[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출
sf_all.loc[c3, 'Series'] = 'CL600'

C:\Users\paul76.lee\AppData\Local\Temp\ipykernel_960\1198681111.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  sf_all.loc[c2, 'Series'] = sf_all[c2]['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).str.replace('D|G|N|W|C', '').apply(lambda x:x[:6]) # PC모델의 대표모델명 추출


In [208]:
sf_all = pd.merge(sf_all, site[['To Site', 'Subsidiary', 'Country']], how='left')
sf_all.loc[sf_all['Division']=='PC', 'SCM'] = sf_all.loc[sf_all['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='PC', 'MKT'] = sf_all.loc[sf_all['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='MNT', 'SCM'] = sf_all.loc[sf_all['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='MNT', 'MKT'] = sf_all.loc[sf_all['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
us_b2b = sf_all.loc[sf_all['To Site'].str.contains('ENUS') & (sf_all['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
sf_all.loc[sf_all['To Site'].str.contains('ENUS') & (sf_all['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b
sf_all = sf_all.fillna('N/A')

In [209]:
gap = sf_all.groupby(['Country', 'MKT', 'SCM', 'Planweek', 'Created_at', 'To Site', 'Ver', 'Series', 'Month', 'Week Name'])['QTY'].sum().unstack(['Planweek', 'Ver', 'Created_at']).fillna(0).astype(int)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap = gap.unstack('Month').fillna(0).astype(int)
gap['Sum of Gap'] = gap['Gap'].sum(axis=1)
gap = gap.reset_index()

In [210]:
countries = gap['Country'].unique()

In [211]:
# 국가별 탭으로 엑셀 파일 만들기
df_all = []
for country in countries:
    df = gap[gap['Country'] == country].copy()
    df = df.reset_index(drop=True).set_index(['Country', 'To Site', 'Series', 'Week Name'])
    df_all.append(df)

updated_time = datetime.datetime.now().strftime('%Y%m%d%H%M')
filepath = f'D:/SP검토/{updated_time}_{p_wk}_TCL Sales Forecast Review_by_country.xlsx'
with pd.ExcelWriter(filepath) as writer:
    for i, df in enumerate(df_all):
        df.to_excel(writer, sheet_name=f'{countries[i]}', startrow=1)

In [177]:
wb = load_workbook(filepath)

for sheet_name in wb.sheetnames:
    confirm_wk = '2024-02-12(W07)'
    color = PatternFill(fgColor='9999FF', fill_type='solid') # violet
    confirm_color = PatternFill(fgColor='00cc00', fill_type='solid') # Honeydew
    pseudo_confirm_color = PatternFill(fgColor='9ACD32', fill_type='solid') # Yellowgreen
    border_style = Side(border_style="thin", color="000000")
    border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)
    sh = wb[sheet_name]
    sh['F1'] = 'Sub TTL'
    for i in range(7, sh.max_column+1):
        col_letter = get_column_letter(i)
        sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
        sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
    for row in sh.iter_rows(min_row=2, min_col=6, max_row=5, max_col=sh.max_column):
        for cell in row:
            cell.fill = color
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    merge_range = f'{get_column_letter(sh.max_column)}2:{get_column_letter(sh.max_column)}5'
    sh.merge_cells(merge_range)
    for cell in sh[merge_range][0]:
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    for row in sh.iter_rows(min_row=6, min_col=1, max_row=6, max_col=6):
        for cell in row:
            cell.fill = color
    for row in sh.iter_rows(min_row=7, min_col=7, max_row=sh.max_row, max_col=sh.max_column):
        for cell in row:
            cell.border = border
            cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
    for row in sh.iter_rows(min_row=7, min_col=6):
        if row[0].value == confirm_wk:
            for cell in row:
                cell.fill = confirm_color
        elif row[0].value in [get_weekname_from(confirm_wk, i) for i in range(1, 3)]:
            for cell in row:
                cell.fill = pseudo_confirm_color
    sh.freeze_panes = 'G7'
    sh.column_dimensions['A'].width = 15
    sh.column_dimensions['B'].width = 6
    sh.column_dimensions['C'].width = 6
    sh.column_dimensions['F'].width = 17.5
    sh.sheet_view.showGridLines = False

In [178]:
wb.save(filepath)
wb.close()

### 2) 업체별로 볼 때

In [26]:
ver_table = sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('SP').reset_index()
ver_table = ver_table[ver_table['From Site'].str.contains('TCL')]
ver_table.iloc[-30:, ]

,Planweek,From Site,Ver,Created_at,QTY
3,2024-01-08(W02),OS_TCL_CN_P,Final,2024010809,372706
9,2024-01-15(W03),OS_TCL_CN_P,Final,2024011509,410525
15,2024-01-22(W04),OS_TCL_CN_P,Final,20240122,428499
24,2024-01-29(W05),OS_TCL_CN_P,Final,2024013016,436464
25,2024-01-29(W05),OS_TCL_CN_P,Main,2024013009,448394


In [28]:
i1 = 24
i2 = 15

p_wk = sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Planweek']
vendor = sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['From Site'].split('_')[1]
ver = sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('SP').reset_index().iloc[i1, :4]['Ver']

In [29]:
sf1 = sf.merge(sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('QTY').reset_index().iloc[[i1], :4])
sf2 = sf.merge(sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('QTY').reset_index().iloc[[i2], :4])
sf2 = sf2[sf2['Week Name'] >= sf1['Week Name'].min()]

In [30]:
sf_all = pd.concat([sf1, sf2])
sf_all['Month'] = sf_all['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')

In [31]:
sf_all = sf_all[sf_all['Week Name'] >= [get_weekname_from(p_wk, i) for i in range(-3, 1)][0]] # 이번주 기준 3주전부터 Sorting
sf_all = sf_all[sf_all['Month'] <= '2024-05'] # 24년 4월까지만 한정

In [32]:
sf_all.insert(7, 'Series', sf_all['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]))
sf_all.insert(7, 'Inch', sf_all['Mapping Model.Suffix'].apply(lambda x:x[:2]))
sf_all = pd.merge(sf_all, site[['To Site', 'Subsidiary', 'Country']], how='left')

In [33]:
sf_all.loc[sf_all['Division']=='PC', 'SCM'] = sf_all.loc[sf_all['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='PC', 'MKT'] = sf_all.loc[sf_all['Division']=='PC', 'Subsidiary'].map(cont[(cont['Div']=='PC') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='MNT', 'SCM'] = sf_all.loc[sf_all['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='SCM')].groupby(['Subsidiary'])['Name'].max())
sf_all.loc[sf_all['Division']=='MNT', 'MKT'] = sf_all.loc[sf_all['Division']=='MNT', 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].max())
us_b2b = sf_all.loc[sf_all['To Site'].str.contains('ENUS') & (sf_all['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'Subsidiary'].map(cont[(cont['Div']=='MNT') & (cont['Team']=='MKT')].groupby(['Subsidiary'])['Name'].min())
sf_all.loc[sf_all['To Site'].str.contains('ENUS') & (sf_all['Mapping Model.Suffix'].apply(lambda x:x[2]) == 'B'), 'MKT'] = us_b2b
sf_all = sf_all.fillna('N/A')

In [34]:
gap = sf_all.groupby(['Country', 'MKT', 'SCM', 'Planweek', 'Created_at', 'To Site', 'Ver', 'Series', 'Month', 'Week Name'])['QTY'].sum().unstack(['Planweek', 'Ver', 'Created_at']).fillna(0).astype(int)
gap['Gap'] = gap[gap.columns[-1]] - gap[gap.columns[0]]
gap = gap.unstack('Month').fillna(0).astype(int)
gap['Sum of Gap'] = gap['Gap'].sum(axis=1)
gap = gap.reset_index()

In [35]:
countries = gap['Country'].unique()

In [36]:
# 국가별 탭으로 엑셀 파일 만들기
df_all = []
for country in countries:
    df = gap[gap['Country'] == country].copy()
    df = df.reset_index(drop=True).set_index(['Country', 'MKT', 'SCM', 'To Site', 'Series', 'Week Name'])
    df_all.append(df)

updated_time = datetime.datetime.now().strftime('%Y%m%d%H%M')
filepath = f'D:/SP검토/{updated_time}_{p_wk}_TCL Sales Forecast Review_by_country.xlsx'
with pd.ExcelWriter(filepath) as writer:
    for i, df in enumerate(df_all):
        df.to_excel(writer, sheet_name=f'{countries[i]}', startrow=1)

## TCL 최근 5주간 변동 현황

In [37]:
ver_table.iloc[-30:, ]

,Planweek,From Site,Ver,Created_at,QTY
3,2024-01-08(W02),OS_TCL_CN_P,Final,2024010809,372706
9,2024-01-15(W03),OS_TCL_CN_P,Final,2024011509,410525
15,2024-01-22(W04),OS_TCL_CN_P,Final,20240122,428499
24,2024-01-29(W05),OS_TCL_CN_P,Final,2024013016,436464
25,2024-01-29(W05),OS_TCL_CN_P,Main,2024013009,448394


In [38]:
idxs = [3, 9, 15, 24] # index 넘버를 선택
df = pd.DataFrame()
for i in idxs:
    df = pd.concat([df, sf.merge(sf.groupby(['Planweek', 'From Site', 'Ver', 'Created_at']).sum('QTY').reset_index().iloc[[i], :-1])]).reset_index(drop=True)    

In [39]:
search_period = ['2024-02', '2024-03', '2024-04', '2024-05', '2024-06']

In [40]:
df['Month'] = df['Week Name'].apply(lambda x:f'{datetime.date.fromisoformat(x[:10]).isocalendar().year}-{get_month_from_date(datetime.date.fromisoformat(x[:10])):02d}')
df.insert(7, 'Series', df['Mapping Model.Suffix'].apply(lambda x:x.split('-')[0]).apply(lambda x:x.split('.')[0]))
df.insert(7, 'Inch', df['Mapping Model.Suffix'].apply(lambda x:x[:2]))
df = df[df['Month'].isin(search_period)]

df = pd.merge(df, site[['To Site', 'Subsidiary', 'Country']], how='left')

In [41]:
wb = load_workbook(filepath)

for sheet_name in wb.sheetnames:
    confirm_wk = '2024-02-12(W07)'
    color = PatternFill(fgColor='9999FF', fill_type='solid') # violet
    confirm_color = PatternFill(fgColor='00cc00', fill_type='solid') # Honeydew
    pseudo_confirm_color = PatternFill(fgColor='9ACD32', fill_type='solid') # Yellowgreen
    border_style = Side(border_style="thin", color="000000")
    border = Border(top=border_style, right=border_style, bottom=border_style, left=border_style)
    sh = wb[sheet_name]
    sh['F1'] = 'Sub TTL'
    for i in range(7, sh.max_column+1):
        col_letter = get_column_letter(i)
        sh.cell(row=1, column=i).value = f'=subtotal(9, {col_letter}7:{col_letter}{sh.max_row})'
        sh.cell(row=1, column=i).number_format = '#,##0;[RED]Δ#,##0;"-"'
    for row in sh.iter_rows(min_row=2, min_col=6, max_row=5, max_col=sh.max_column):
        for cell in row:
            cell.fill = color
            cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    merge_range = f'{get_column_letter(sh.max_column)}2:{get_column_letter(sh.max_column)}5'
    sh.merge_cells(merge_range)
    for cell in sh[merge_range][0]:
        cell.alignment = Alignment(horizontal='center', vertical='center', wrap_text=True)
    for row in sh.iter_rows(min_row=6, min_col=1, max_row=6, max_col=6):
        for cell in row:
            cell.fill = color
    for row in sh.iter_rows(min_row=7, min_col=7, max_row=sh.max_row, max_col=sh.max_column):
        for cell in row:
            cell.border = border
            cell.number_format = '#,##0;[RED]Δ#,##0;"-"'
    for row in sh.iter_rows(min_row=7, min_col=6):
        if row[0].value == confirm_wk:
            for cell in row:
                cell.fill = confirm_color
        elif row[0].value in [get_weekname_from(confirm_wk, i) for i in range(1, 3)]:
            for cell in row:
                cell.fill = pseudo_confirm_color
    sh.freeze_panes = 'G7'
    sh.column_dimensions['A'].width = 15
    sh.column_dimensions['B'].width = 6
    sh.column_dimensions['C'].width = 6
    sh.column_dimensions['F'].width = 17.5
    sh.sheet_view.showGridLines = False
    
    df_bar = df.groupby(['Country', 'Planweek', 'Series', 'Month']).sum('QTY').loc[sheet_name].reset_index()
    fig = px.bar(df_bar, x='Planweek', y='QTY', color='Month', facet_col='Series', title=f'{sheet_name} Sales Forecast Variation',
                 category_orders={'Month':['2024-02', '2024-03', '2024-04', '2024-05', '2024-06'], 'Planweek':sorted(df['Planweek'].unique())}, width=1200, text_auto=True)
    image_filepath = f'D:/figure/{sheet_name}_weekly_sales_forecast_variation.png'
    fig.write_image(image_filepath)
    
    plot_image = Image(image_filepath)
    sh.add_image(plot_image, f'A{sh.max_row+2}')

wb.save(filepath)
wb.close()